In [1]:
# http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#association-object
import sqlalchemy
sqlalchemy.__version__ 

'1.2.4'

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [3]:
from sqlalchemy import Column, Integer, String
from sqlalchemy import Table, ForeignKey
from sqlalchemy.orm import relationship

class Association(Base):
    __tablename__ = 'association'
    left_id = Column(Integer, ForeignKey('left.id'), primary_key=True)
    right_id = Column(Integer, ForeignKey('right.id'), primary_key=True)
    extra_data = Column(String(50))
    child = relationship("Child", back_populates="parents")
    parent = relationship("Parent", back_populates="children")

class Parent(Base):
    __tablename__ = 'left'
    id = Column(Integer, primary_key=True)
    children = relationship("Association", back_populates="parent")

class Child(Base):
    __tablename__ = 'right'
    id = Column(Integer, primary_key=True)
    parents = relationship("Association", back_populates="child")
    
Base.metadata.create_all(engine)

2018-03-08 00:47:56,005 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-03-08 00:47:56,006 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:47:56,008 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-03-08 00:47:56,008 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:47:56,009 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("association")
2018-03-08 00:47:56,011 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:47:56,013 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("left")
2018-03-08 00:47:56,014 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:47:56,015 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("right")
2018-03-08 00:47:56,015 INFO sqlalchemy.engine.base.Engine ()
2018-03-08 00:47:56,016 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "left" (
	id INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2018-03-08 00:47:56,017 INFO sqlalchemy.engine.base.Engine ()


In [4]:
s = Session()

child = Child(id = 10)
parent = Parent(id = 20)

ass = Association(extra_data="HELLO WORLD")  # (_*_) 
ass.child = child
ass.parent = parent

s.add(child)
s.commit()  # this will commit all of child, parent and ass

2018-03-08 00:47:56,043 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:47:56,045 INFO sqlalchemy.engine.base.Engine INSERT INTO "left" (id) VALUES (?)
2018-03-08 00:47:56,046 INFO sqlalchemy.engine.base.Engine (20,)
2018-03-08 00:47:56,047 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (?)
2018-03-08 00:47:56,047 INFO sqlalchemy.engine.base.Engine (10,)
2018-03-08 00:47:56,049 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id, extra_data) VALUES (?, ?, ?)
2018-03-08 00:47:56,050 INFO sqlalchemy.engine.base.Engine (20, 10, 'HELLO WORLD')
2018-03-08 00:47:56,054 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
for i in range(5):
    s = Session()
    
    child = Child(id = i)

    ass = Association(extra_data="HELLO WORLD")  # (_*_) 
    ass.child = child
    ass.parent = s.query(Parent).first()

    s.add(child)
    s.commit()  # this will commit all of child, parent and ass
    

2018-03-08 00:47:56,067 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:47:56,068 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
 LIMIT ? OFFSET ?
2018-03-08 00:47:56,069 INFO sqlalchemy.engine.base.Engine (1, 0)
2018-03-08 00:47:56,070 INFO sqlalchemy.engine.base.Engine INSERT INTO "right" (id) VALUES (?)
2018-03-08 00:47:56,071 INFO sqlalchemy.engine.base.Engine (0,)
2018-03-08 00:47:56,072 INFO sqlalchemy.engine.base.Engine INSERT INTO association (left_id, right_id, extra_data) VALUES (?, ?, ?)
2018-03-08 00:47:56,074 INFO sqlalchemy.engine.base.Engine (20, 0, 'HELLO WORLD')
2018-03-08 00:47:56,075 INFO sqlalchemy.engine.base.Engine COMMIT
2018-03-08 00:47:56,077 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:47:56,078 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
 LIMIT ? OFFSET ?
2018-03-08 00:47:56,078 INFO sqlalchemy.engine.base.Engine (1, 0)
2018-03-08 00:47:56,080 INFO sqlalchemy.e

In [6]:
# test if extra_data has been insert well
s = Session()
asses = s.query(Association).all()
print([i.extra_data for i in asses])  
s.commit()

2018-03-08 00:47:56,126 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:47:56,127 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data 
FROM association
2018-03-08 00:47:56,128 INFO sqlalchemy.engine.base.Engine ()
['HELLO WORLD', 'HELLO WORLD', 'HELLO WORLD', 'HELLO WORLD', 'HELLO WORLD', 'HELLO WORLD']
2018-03-08 00:47:56,130 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
# test Parent.children 
s = Session()

parent = s.query(Parent).first()

print('===========')
print('====>', [ass.right_id for ass in parent.children])
print('====>', [ass.parent for ass in parent.children])
print('===========')

s.commit()

2018-03-08 00:48:47,421 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-03-08 00:48:47,422 INFO sqlalchemy.engine.base.Engine SELECT "left".id AS left_id 
FROM "left"
 LIMIT ? OFFSET ?
2018-03-08 00:48:47,423 INFO sqlalchemy.engine.base.Engine (1, 0)
2018-03-08 00:48:47,425 INFO sqlalchemy.engine.base.Engine SELECT association.left_id AS association_left_id, association.right_id AS association_right_id, association.extra_data AS association_extra_data 
FROM association 
WHERE ? = association.left_id
2018-03-08 00:48:47,426 INFO sqlalchemy.engine.base.Engine (20,)
====> [0, 1, 2, 3, 4, 10]
====> [<__main__.Parent object at 0x112191048>, <__main__.Parent object at 0x112191048>, <__main__.Parent object at 0x112191048>, <__main__.Parent object at 0x112191048>, <__main__.Parent object at 0x112191048>, <__main__.Parent object at 0x112191048>]
2018-03-08 00:48:47,427 INFO sqlalchemy.engine.base.Engine COMMIT
